<a href="https://colab.research.google.com/github/cle173/Lyric-Generator/blob/main/GPT2_Lyric_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

kaggle/input/taylor_swift_lyrics.csv
kaggle/input/IndianFoodDatasetXLS.xlsx
kaggle/input/IndianFoodDatasetCSV.csv


In [ ]:
!pip3 install aitextgen

In [ ]:
from aitextgen import aitextgen

In [ ]:
! nvidia-smi

Wed May 11 08:01:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    29W /  70W |   1996MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
ai = aitextgen(tf_gpt2="124M", to_gpu=True)

In [ ]:
!ls

aitextgen  input_text_cleaned.txt  kaggle  sample_data	trained_model


In [ ]:
input_file = pd.read_csv('kaggle/input/taylor_swift_lyrics.csv', encoding = "latin1")

In [ ]:
input_file.head()

,artist,album,track_title,track_n,lyric,line,year
0,Taylor Swift,Taylor Swift,Tim McGraw,1,He said the way my blue eyes shined,1,2006
1,Taylor Swift,Taylor Swift,Tim McGraw,1,Put those Georgia stars to shame that night,2,2006
2,Taylor Swift,Taylor Swift,Tim McGraw,1,"I said, ""That's a lie""",3,2006
3,Taylor Swift,Taylor Swift,Tim McGraw,1,Just a boy in a Chevy truck,4,2006
4,Taylor Swift,Taylor Swift,Tim McGraw,1,That had a tendency of gettin' stuck,5,2006


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
input_file.head()

,artist,album,track_title,track_n,lyric,line,year
0,Taylor Swift,Taylor Swift,Tim McGraw,1,He said the way my blue eyes shined,1,2006
1,Taylor Swift,Taylor Swift,Tim McGraw,1,Put those Georgia stars to shame that night,2,2006
2,Taylor Swift,Taylor Swift,Tim McGraw,1,"I said, ""That's a lie""",3,2006
3,Taylor Swift,Taylor Swift,Tim McGraw,1,Just a boy in a Chevy truck,4,2006
4,Taylor Swift,Taylor Swift,Tim McGraw,1,That had a tendency of gettin' stuck,5,2006


In [ ]:
df = pd.DataFrame(input_file["lyric"])

In [ ]:
df.shape

(4862, 1)

In [ ]:
df.head()

,lyric
0,He said the way my blue eyes shined
1,Put those Georgia stars to shame that night
2,"I said, ""That's a lie"""
3,Just a boy in a Chevy truck
4,That had a tendency of gettin' stuck


In [ ]:
df = df.assign(var1=df['lyric'].str.split('-')).explode('var1')

In [ ]:
df.var1 = df.var1.str.lstrip()

In [ ]:
df.head()

,lyric,var1
0,He said the way my blue eyes shined,He said the way my blue eyes shined
1,Put those Georgia stars to shame that night,Put those Georgia stars to shame that night
2,"I said, ""That's a lie""","I said, ""That's a lie"""
3,Just a boy in a Chevy truck,Just a boy in a Chevy truck
4,That had a tendency of gettin' stuck,That had a tendency of gettin' stuck


In [ ]:
df.shape

(5083, 2)

In [ ]:
df.to_csv("input_text_cleaned.txt", columns=["var1"], header=False, index=False)

In [ ]:
ai.train('input_text_cleaned.txt',
         line_by_line=False,
         from_cache=False,
         num_steps=500,
         generate_every=100,
         save_every=500,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1, 
         )

In [ ]:
ai = aitextgen(model_folder="trained_model", config="./trained_model/config.json", to_gpu=True)

In [ ]:
prompts=["Hi Mom", "I love pizza", "I love eat ice-cream", "I love CS5170"]

In [ ]:
for prompt in prompts:
  ai.generate(n=1,
              batch_size=1,
              max_length=50,
              temperature=1.0,
              prompt=prompt,
              top_p=0.9)

Hi Mom's waiting up and you're cursing
I'm so in love that I acted insane
And that's how it works
That's how you get the girl
"Yeah, yeah"
yeah
She's so tall and handsome as
I love pizza and that's how it works
"That's how you get the girl, girl (get the girl (get the girl! That's how it works!)"
"Remind her how it used to be, yeah"

I love eat ice-cream, it's all the same"
"I'm so tired of your attitude, "
I feel so deep
Run your hands through your hair
Absent mindedly making me want you
And I don't
I love CS5170, finding things they never found"
They might be the burning witches even if they aren't one
"The light me up (light me up (light me up), light me up (light me up (light me
